In [5]:
#!/usr/bin/python
## email_preprocess code start here--------------------------------------------------------------------------------##

import pickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "tools/word_data.pkl", authors_file="tools/email_authors.pkl"):

    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = pickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    #selector = SelectPercentile(f_classif, percentile=1)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

#######################################################################################################################

#--------------------SVM-Solution-------------------------------------------------------------------------


#########################################################
### your code goes here ###
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#clf=SVC(kernel='linear')
clf=SVC(kernel='rbf',C=10000.0)
t0=time()
clf.fit(features_train,labels_train)
print("Training time: ",round(time()-t0,3),"secs")
t1=time()
pred=clf.predict(features_test)
#pred=clf.predict([features_test[10],features_test[26],features_test[50]])
print("Prediction time: ",round(time()-t1,3),"secs")
#count_chris=0
#for i in pred:
#    if i==1:
#        count_chris+=1
#print(count_chris)
print ("Accuracy After using: SVM ",accuracy_score(pred,labels_test))
print("------- OR ---------")
print(accuracy_score(labels_test,pred)*100)

#########################################################

Training time:  122.012 secs
Prediction time:  12.507 secs
Accuracy After using: SVM  0.9908987485779295
------- OR ---------
99.08987485779295
